In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [4]:
# read in the datasets
 
#jd = pd.read_csv('job_data_dict_gt30.csv')

# this is a data set of 177 with exactly 30 randomly sample each
# jd = pd.read_csv('job_data_dict_30_sample.csv')

# this is a data set with 177 job titles with at least 30 observations each and the 
# 'jobtitle' words occuring twice in the dictionary
jd = pd.read_csv('job_data_dict_all_weighted.csv')

In [5]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 43336 entries, 0 to 43335
Data columns (total 5 columns):
Unnamed: 0    43336 non-null int64
jobs          43336 non-null object
jobtitle      43336 non-null object
dictionary    43336 non-null object
id            43336 non-null int64
dtypes: int64(2), object(3)
memory usage: 2.0+ MB


In [6]:
jd.head()

,Unnamed: 0,jobs,jobtitle,dictionary,id
0,1,supervisors of office and administrative staff,client success account coordinator,client success account coordinator client succ...,1
1,2,logisticians,account coordinator supply chain customer service,account coordinator supply chain customer serv...,2
2,3,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...,3
3,4,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...,4
4,5,sales reps wholesale and manufacturing,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...,5


In [7]:

cat_columns = ['jobs','jobtitle','dictionary']
feature_columns = ['jobtitle','dictionary']   
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,jobs,jobtitle,dictionary
0,supervisors of office and administrative staff,client success account coordinator,client success account coordinator client succ...
1,logisticians,account coordinator supply chain customer service,account coordinator supply chain customer serv...
2,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
3,personnel recruiters,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
4,sales reps wholesale and manufacturing,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...


In [8]:
jd_2.jobs.value_counts().min()

30

In [9]:
df_numerical = jd_2.copy()
encoder_dict = {}

for col in string_variables:
    le = preprocessing.LabelEncoder()
    le.fit(jd_2[col])
    encoder_dict[col] = le
    df_numerical[col] = le.transform(jd_2[col])

df_numerical.head()

,jobs,jobtitle,dictionary
0,157,client success account coordinator,client success account coordinator client succ...
1,89,account coordinator supply chain customer service,account coordinator supply chain customer serv...
2,119,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
3,119,staffing sales account manager coordinator rec...,staffing sales account manager coordinator rec...
4,144,sales coordinator inside representative accoun...,sales coordinator inside representative accoun...


In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_2['dictionary'], jd_2['jobs'], random_state=1)

In [11]:
feat = list(df_numerical.dictionary.values.flatten())
type(feat)
#feat = unicode(feat, 'utf-8')

list

In [12]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
y_test


array(['medical secretaries', 'travel agents',
       'education administrators postsecondary', ..., 'production workers',
       'secretaries not legal or medical',
       'demonstrators and product promoters'], dtype=object)

In [13]:
X_test.shape

(10834L,)

In [14]:
X_train.shape

(32502L,)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [16]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

<32502x14392 sparse matrix of type '<type 'numpy.int64'>'
	with 343223 stored elements in Compressed Sparse Row format>

In [17]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

<10834x14392 sparse matrix of type '<type 'numpy.int64'>'
	with 112458 stored elements in Compressed Sparse Row format>

In [18]:
train_features = vect.get_feature_names()
len(train_features)

14392

In [20]:
#train_features[:50]

In [22]:
#train_features[-50:]

In [23]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_arr, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
7644,15392,manager
11563,12964,senior
3705,10005,director
4321,8440,engineer
11278,8136,sales
1003,6013,assistant
1783,5634,business
7717,5377,marketing
11616,4930,service
8963,4865,operations


In [25]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [26]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm)
preds

array(['supervisors of office and administrative staff', 'travel agents',
       'medical and health services managers', ...,
       'supervisors of production and operating staff',
       'supervisors of office and administrative staff',
       'demonstrators and product promoters'], 
      dtype='|S88')

In [27]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.612977662913
[[194   0   0 ...,   0   0   0]
 [  0 132   0 ...,   0   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [  0   0   0 ..., 144   0   0]
 [  0   0   0 ...,   0  10   0]
 [  0   0   0 ...,   0   0   0]]


In [28]:
# predict (poorly calibrated) probabilities and calculate AUC
probs = nb.predict_proba(test_dtm)[:, 1]
probs
#print(metrics.roc_auc_score(y_test, probs))

array([  2.19064044e-06,   1.22452450e-15,   2.53323817e-09, ...,
         5.14408452e-08,   3.65125321e-07,   2.04739789e-07])

In [25]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [26]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [29]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

accountants       0.68      0.83      0.75       233
administrative assistants       0.66      0.78      0.72       170
administrative services managers       0.00      0.00      0.00         8
advertising and promotions managers       0.00      0.00      0.00         8
advertising sales agents       0.93      0.33      0.49        42
art directors       0.00      0.00      0.00        18
assemblers and fabricators       0.75      0.35      0.48        17
audio and video equipment technicians       0.50      0.12      0.20         8
   auditors       0.92      0.63      0.75        52
automotive master mechanics       0.83      0.50      0.62        10
automotive specialty technicians       0.89      0.62      0.73        13
 bartenders       0.83      0.77      0.80        13
bill and account collectors       0.84      0.86      0.85        49
billing cost and rate clerks       0.79      0.51      0.62        37
biological technici

C:\Anaconda\lib\site-packages\sklearn\metrics\metrics.py:1771: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [30]:

target_predicted = nb.predict(test_dtm)
target_prob = nb.predict_proba(test_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(y_test, target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,advertising sales agents,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,automotive specialty technicians,bartenders,bill and account collectors,...,telemarketers,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,194,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,233
administrative assistants,0,132,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,170
administrative services managers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
advertising and promotions managers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
advertising sales agents,0,0,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,42
art directors,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
assemblers and fabricators,0,0,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
audio and video equipment technicians,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
auditors,4,0,0,0,0,33,0,0,0,0,...,0,0,1,0,0,0,0,0,0,52


In [29]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, y_train, cv=5, scoring='accuracy').mean()

0.40240882638027997

In [ ]:
# try using all of the data

In [32]:
train_dtm = vect.fit_transform(jd_2['dictionary'])


In [33]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm, jd_2['jobs'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
preds = nb.predict(train_dtm)
preds.shape

(43336L,)

In [35]:

print(classification_report(jd_2['jobs'], preds) )

             precision    recall  f1-score   support

accountants       0.79      0.89      0.83       881
administrative assistants       0.76      0.88      0.82       657
administrative services managers       0.00      0.00      0.00        53
advertising and promotions managers       1.00      0.02      0.04        46
advertising sales agents       0.97      0.49      0.65       149
art directors       1.00      0.04      0.09        67
assemblers and fabricators       0.73      0.65      0.68        79
audio and video equipment technicians       1.00      0.41      0.58        32
   auditors       0.97      0.75      0.85       220
automotive master mechanics       0.84      0.72      0.77        43
automotive specialty technicians       0.87      0.73      0.80        56
 bartenders       0.96      0.83      0.89        59
bill and account collectors       0.90      0.85      0.88       246
billing cost and rate clerks       0.95      0.67      0.79       171
biological technici

In [36]:

target_predicted = nb.predict(train_dtm)
target_prob = nb.predict_proba(train_dtm)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_2['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,administrative assistants,advertising and promotions managers,advertising sales agents,art directors,assemblers and fabricators,audio and video equipment technicians,auditors,automotive master mechanics,automotive specialty technicians,...,travel agents,treasurers and controllers,truck drivers heavy and tractor trailer,truck drivers light or delivery services,vocational education teachers postsecondary,waiters and waitresses,web developers,welders cutters and welder fitters,wholesale and retail buyers except farm products,All
True,,,,,,,,,,,,,,,,,,,,,
accountants,782,2,0,0,0,0,0,0,1,0,...,0,2,0,0,0,0,0,0,0,881
administrative assistants,0,576,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,657
administrative services managers,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,53
advertising and promotions managers,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,46
advertising sales agents,0,0,0,73,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,149
art directors,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,67
assemblers and fabricators,0,0,0,0,0,51,0,0,0,0,...,0,0,0,0,0,0,0,0,0,79
audio and video equipment technicians,0,0,0,0,0,0,13,0,0,0,...,0,0,0,0,0,0,0,0,0,32
auditors,11,0,0,0,0,0,0,165,0,0,...,0,1,0,0,0,0,0,0,0,220


In [37]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, jd_2['jobs'], cv=5, scoring='accuracy').mean()

0.54515312834797669